# TEI Conference 2019 #

## Etape 3 : L'Analyse ##

### Préambule ###

L'objectif présent est d'analyser les abstracts afin de les regrouper en cluster, permettant de dégager des thématiques, de comparer les différentes conférences ou bien d'étudier si les distinctions faites par les organisateurs de la conférence 2019 entre les différentes interventions est bien pertinente.
Nous pourrons également critiquer les choix de l'ordinateur.

Nous allons faire ici de la classification non supervisée grâce au package SKLearn, librairie dédiée au Machine Learning notamment.

### Les packages ### 


Il faut lancer la cellule ci-dessous pour importer les packages dédiés.

La librairie SKLearn permet, grâce à TF-IdF, de vectoriser les textes, c'est à dire d'attribuer à chaque mot un vecteur unique le caractérisant dans l'espaces. KMeans permet en outre de définir des clusters.

La librairie spaCy permet de lemmatiser, tandis que la librairie PorterStemmer de NLTK permet de stemmer.

Les librairies Pandas, Numpy et Scipy permettent de traiter la donnée et de produire des tableaux à partir des textes vectorisés.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import os
import csv
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize

#from gensim.models import ldamodel #utiliser gensim pour du topic modeling
#import gensim.corpora

#le vrai Topic Modelling, c'est LDA pas NMF comme j'ai fait en bas
# Au lieu d'utiliser le tf idf
#La sémantique distributionnelle: "You know the sense of a word with the company it have."
#Pour mon mémoire, lire des trucs sur Word2Vec, voir d'où vient le gars et pourquoi ça incide sur mon travail
#FastText aussi, l'étudier, voir de quel entreprise ça vient
#lire l'ouvrage Ancient Manuscript in Digital Culture, lire l'article "Qualitativ Analysis of Semantic Linguage Models"
#Lire le 1, le 2.1, (le 2.2 par curiosité). C'est simple et accessible, lire les concepts généraux qu'il donne et utiliser la biblio
#ensuite, j'irai lire les articles publiés en 2019. Obligé de le lire pour le mémoire.

### Choisir son corpus de travail ###

Ensuite, il faut choisir quel set sera loadé et analysé entre les abstracts lemmatisés et les abstracts loadés. Le résultat change entre les deux versions.

In [5]:
#Pour utiliser les textes lemmatisés, c'est ici

documents = []
Path = "./cache2019/cacheLEM/"
filelist = os.listdir(Path) #filelist est une liste regroupant tous les chemins vers les différents abstracts.

for abstract in filelist:
    with open(Path + abstract, "r", encoding="UTF-8") as y:
        texte = y.read()
        documents.append(texte)

In [ ]:
#Pour utiliser les textes stemmés, c'est ici

documents = []
Path = "./cache2019/cacheSTEM/"
filelist = os.listdir(Path) #filelist est une liste regroupant tous les chemins vers les différents abstracts.

for abstract in filelist:
    with open(Path + abstract, "r", encoding="UTF-8") as y:
        texte = y.read()
        documents.append(texte)





## Définir les clusters ##

Après avoir tokennisé et lemmatisé/stemmé nos abstracts, nous allons maintenant les vectoriser (représenter chaque mot sous la forme d'un vecteur sur un plan ordonné en deux dimensions) puis ensuite nous allons laisser la machine définir des clusters, c'est à dire des groupes de mots qui ont un sens commun.

Ici, on associe à chaque mot de tous les abstracts une coordonnée unique à chaque mot utilisé, pour ensuite pouvoir les placer sur un plan en 2D et ainsi relever des ressemblances.

In [6]:
#vectorize the text i.e. convert the strings to numeric features
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

Ensuite, on réunit les termes vectorisés en cluster, c'est à dire en groupe de ressemblance.

In [7]:
#cluster documents, ici 8 clusters
true_k = 8
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=8, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

Enfin, on montre les termes que l'ordinateur a pu rassembler ensemble.

In [8]:
#print top terms per cluster clusters
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
        

        


Top terms per cluster:
Cluster 0:
 textual
 pron
 letter
 rochlitz
 text
 stance
 artifact
 notational
 variance
 theory
Cluster 1:
 tei
 text
 format
 pron
 edition
 graph
 xml
 performance
 2019
 guidelines
Cluster 2:
 pron
 tei
 use
 element
 xml
 odd
 line
 project
 datum
 edition
Cluster 3:
 description
 watermark
 paper
 pron
 object
 punctuation
 manuscript
 element
 editor
 manuscripta
Cluster 4:
 correspondence
 bibliographic
 cmif
 tei
 pron
 correspsearch
 metadata
 jtei
 frbr
 letter
Cluster 5:
 text
 pron
 network
 indic
 tibetan
 collection
 markup
 corpus
 history
 sarit
Cluster 6:
 token
 structuring
 type
 stage
 sales
 catalogues
 tei
 pron
 direction
 corpus
Cluster 7:
 parliamentary
 clarin
 pp
 parla
 comment
 applause
 interjection
 encode
 austrian
 corpus


## Cluster de documents ##

Grâce à un système de prédiction découlant de l'apprentissage et de l'entraînement sur les termes des abstracts, nous pouvons classifier les abstracts en 8 catégories. Nous pourrons ensuite les comparer avec le classement déjà fait par les organisateurs de la conférence TEI 2019.

In [9]:
Path = "./cache2019/cacheLEM/" #On peut remplacer le cache STEM par LEM à la place, pour ainsi comparer les deux sorties.
filelist = os.listdir(Path) #filelist est une liste regroupant tous les chemins vers les différents abstracts.
liste_triee = [] #c'est la liste contenant les infos triées de tous les abstracts


for abstract in filelist:
    reference = abstract.replace('.txt', '') #je normalise le nom du texte pour le faire correpondre avec celui indiqué dans le tableau csv réunissant toutes les informations
    with open(Path + abstract, "r", encoding="UTF-8") as y:
        liste_resultat_unitaire = [] #j'instancie la liste des infos propres à chaque abstract
        texte = y.read()
        X = vectorizer.transform([texte]) #je vectorise le texte de l'abstract
        predicted = model.predict(X)  #j'attribue l'abstract à un cluster. predicted est de la classe numpy array : c'est un vecteur
        with open('./cache2019/TEI2019.csv', 'r') as csvfile:
            read = csv.reader(csvfile, delimiter = ',')
            titre = ''
            auteurs = ''
            for row in read:
                if row[4] == reference:
                    titre = row[2]
                    auteurs = row[0]
        liste_resultat_unitaire.append(titre)
        liste_resultat_unitaire.append(auteurs)
        liste_resultat_unitaire.append(int(predicted))
        liste_triee.append(liste_resultat_unitaire)
        

print("Dans le groupe 1, il y a :")
for elem in liste_triee:
    if elem[2] == 0:
        print("\t", elem[0] )
        
print("\n Dans le groupe 2, il y a :")
for elem in liste_triee:
    if elem[2] == 1:
        print("\t", elem[0] )
        
print("\n Dans le groupe 3, il y a :")
for elem in liste_triee:
    if elem[2] == 2:
        print("\t", elem[0] )

print("\n Dans le groupe 4, il y a :")
for elem in liste_triee:
    if elem[2] == 3:
        print("\t", elem[0] )

print("\n Dans le groupe 5, il y a :")
for elem in liste_triee:
    if elem[2] == 4:
        print("\t", elem[0] )
        
print("\n Dans le groupe 6, il y a :")
for elem in liste_triee:
    if elem[2] == 5:
        print("\t", elem[0] )
        
print("\n Dans le groupe 7, il y a :")
for elem in liste_triee:
    if elem[2] == 6:
        print("\t", elem[0] )
        
print("\n Dans le groupe 8, il y a :")
for elem in liste_triee:
    if elem[2] == 7:
        print("\t", elem[0] )


#Des librairies python qui convertissent en table HMTL, faut que je fasse de la dataviz
# chord visualisation est une librairie faisant des dataviz, Seaborn aussi, BOKEH aussi, Plotly (MatPlotLib est la base de tous), Parallel Plot est cool.
# On peut utiliser XSLT via LXML

Dans le groupe 1, il y a :
	 Genesis and Variance: From Letter to Literature
	 A realistic theory of textuality and its consequences on digital text representation

 Dans le groupe 2, il y a :
	 TEI XML and Delta Format Interchangeability
	 Reconceiving TEI models of theatrical performance text with reference to promptbooks
	 Making Linkable Data from Account Books: Bookkeeping Ontology in the Digital Edition Publishing Cooperative for Historical Accounts
	 Refining the Current Teaching Methodology of the TEI through the Analysis of Server Logs
	 An Attempt of Dissemination of TEI in a TEI-underdeveloped country: Activities of the SIG EAJ
	 Text Graph Ontology. A Semantic Web approach to represent genetic scholarly editions
	 Using Microsoft Word for preparing XML TEI-compliant digital editions
	 Creating high-quality print from TEI documents

 Dans le groupe 3, il y a :
	 Getting Along with Relational Databases
	 Analyzing and Visualizing Uncertain Knowledge: Introducing the PROVIDEDH

Voici le plan et l'agencement des conférences tel qu'il est indiqué ici : https://graz-2019.tei-c.org/wp-content/uploads/2019/09/ProgrammheftFINAL.pdf

TEI, formal ontologies,controlled vocabularies and Linked OpenData :
- Making Linkable Data from Account Books: Bookkeeping Ontology in the Digital Edition Publishing Cooperative for Historical Accounts
- Inscriptions, Hieroglyphs, Linguistics and Beyond! The Corpus of Classic Mayan as an Ontological Information Resource
- Modeling FRBR Entities and their Relationships with TEI: A Look at HallerNet Bibliographic Descriptions
- Referencing an editorial ontology from the TEI: An attempt to overcome informal typologies
- Text Graph Ontology. A Semantic Web approach to represent genetic scholarly editions

TEI and models of text :
- A realistic theory of textuality and its consequences on digital text representation
- Genesis and Variance: From Letter to Literature
- Between freedom and formalisation: a hypergraph model for representing the nature of text
- Reflecting the Influence of Technology on Models of Text in Scholarly Digital Editing
- Introducing Objectification: when is an <object> a <place> ?
- An Encoding Strategic Proposal of “Ruby” Texts: Examples from Japanese Texts
- Referencing annotations as a core concept of the hallerNet edition and research platform
- Recreating history through events
- Document Modeling with the TEI Critical Apparatus
- Exploring TEI structures to find distinctive features of text types
- Reconceiving TEI models of theatrical performance text with reference to promptbooks
- What is a Line? Encoding and Counting Lines in Early Modern Dramatic Texts
    
TEI across corpora,languages, and cultures :
- Growing collections of TEI texts: Some lessons from SARIT
- Towards larger corpora of Indic texts: For now, minimize metatext
- Encoding history in TEI: A corpus-oriented approach for investigating Tibetan historiography
- Advantages and challenges of tokenized TEI
- A sign of the times: medieval punctuation, its encoding and its rendition in modern times

TEI annotation and publication :
- Analyzing and Visualizing Uncertain Knowledge: Introducing the PROVIDEDH Open Science Platform
- The Prefabricated Website: Who Needs a Server Anyway?
- correspSearch v2 –New ways of exploring correspondence
- Validating @selector: a regular expression adventure
- TEI encoding of correspondence: A community effort

TEI simplification and extension :
- Opportunities and challenges of the TEI for scholarly journals in the Humanities
- Archiving a TEI project FAIRly
- Creating high-quality print from TEI documents
- Native-TEI dialectal dictionary for Bavarian dialects in Austria: data structure, software and workflow
- An Attempt of Dissemination of TEI in a TEI-underdeveloped country: Activities of the SIG EAJ
- Refining the Current Teaching Methodology of the TEI through the Analysis of Server Logs
- Using Github and its Integrations to Create, Test, and Deploy a Digital Edition

TEI environments and infrastructures :
- Parla-CLARIN: TEI guidelines for corpora of parliamentary proceedings
- Challenges in encoding parliamentary data: between applause and interjections
- A TEI customization for the description of paper and watermarks
- How we tripled our encoding speed in the Digital Victorian Periodical Project
- Manuscripta-The editor from past to future
- Highlighting Our Examples: encoding XML examples in pedagogical contexts
- Case Study TEI Customization: A Restricted TEI Format for Edition Open Access (EOA)
- In search of comity: TEI for distant reading

TEI and beyond :interactions, interchange, integrations and interoperability :
- Using Machine Learning for the Automated Classification of Stage Directions in TEI-Encoded Drama Corpora
- TEI XML and Delta Format Interchangeability

TEI and non-XML technologies :
- Five Centuries of History in a Network
- Introducing an Open, Dynamic and Efficient Lexical Data Access for TEI-encoded Dictionaries on the Internet
- Getting Along with Relational Databases
- Using Microsoft Word for preparing XML TEI-compliant digital editions
- Scaling up Automatic Structuring of Manuscript Sales Catalogues

In [10]:
# ne pas le lancer, je stock ici l'exemple pour clusteriser des documents
for abstract in documents:
    X = vectorizer.transform([abstract])
    predicted = model.predict(X)

### Topic Modelling ###

Ensuite, je vais tenter de déterminer des sujets aux cluster, c'est à dire déterminer grâce aux mots de chaque cluster des thèmes propres à chaque cluster grâce à une étude de chacun des termes.

In [11]:
transformer = TfidfTransformer(smooth_idf=False)
x_tfidf = transformer.fit_transform(X)
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

/home/formation1/Documents/Cours/dossier_python/venv/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1466: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [12]:
num_topics = 8 #définition du nombre de topics
model = NMF(n_components=num_topics) 
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=8, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)